# TODO: REMOVE VERIZON!!!

In [1]:
import pandas as pd
import numpy as np
import researchpy # pip install researchpy
import statistics

import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)
import plotly.io as pio
pio.orca.config.executable = '/Users/kjo/anaconda3/bin/orca'
%config InlineBackend.figure_format = 'svg'


carrierDF=pd.read_csv('../data/allCarrierData_final.csv')
carrierDF.Manufacture=carrierDF.Manufacture.str.replace("Samgung","Samsung")
carrierDF.difference=carrierDF.difference.astype('float')


carrierDF=carrierDF[carrierDF.Carrier!="Verizon"] # remove verizon


#save the figures?
saveFigs=True

print(carrierDF.head())

  Carrier Manufacture               Model Release_Date Bulletin_Level  \
0    AT&T        ASUS      ASUS PadFone X   2015-12-30     2015-12-01   
1    AT&T     Alcatel  Alcatel IdealXCITE   2018-04-03     2018-02-01   
2    AT&T     Alcatel  Alcatel IdealXCITE   2018-10-24     2018-08-01   
3    AT&T     Alcatel  Alcatel IdealXCITE   2018-12-18     2018-10-01   
4    AT&T  BlackBerry   BlackBerry KEYone   2017-09-26     2017-09-01   

   difference Android_Level_Guessed  \
0        23.0                  True   
1        57.0                 False   
2        79.0                 False   
3        78.0                 False   
4        21.0                 False   

                                               build  Year_Phone_Release  \
0              M3.15.53.P5 ATT_11.10.11.1 11.10.11.1                2014   
1        JO.2.0.c1-00150-8909_GEN_PACK-1 nan YAMUAM0                2017   
2        JO.2.0.c1-00267-8909_GEN_PACK-2 nan YAYUAY0                2017   
3        JO.2.0.c1-002

In [8]:
print(carrierDF[(carrierDF.Manufacture=="Google")])

      Carrier Manufacture        Model Release_Date Bulletin_Level  \
20       AT&T      Google      Pixel 4   2019-12-16     2019-11-01   
21       AT&T      Google   Pixel 4 XL   2019-12-16     2019-11-01   
409    Sprint      Google      Pixel 3   2019-05-08     2019-05-01   
410    Sprint      Google   Pixel 3 XL   2019-05-08     2019-05-01   
773  T-Mobile      Google      Pixel 3   2019-07-26     2019-08-01   
774  T-Mobile      Google   Pixel 3 XL   2019-07-26     2019-08-01   
775  T-Mobile      Google     Pixel 3a   2019-05-30     2019-05-01   
776  T-Mobile      Google     Pixel 3a   2019-07-26     2019-08-01   
777  T-Mobile      Google  Pixel 3a XL   2019-05-30     2019-05-01   
778  T-Mobile      Google  Pixel 3a XL   2019-07-26     2019-08-01   

     difference Android_Level_Guessed  \
20         42.0                   NaN   
21         42.0                   NaN   
409         2.0                 False   
410         2.0                 False   
773       -10.0         

In [2]:
lvlColor={}
carrierColors={'AT&T':[64, 190, 245],'Sprint':[255,162,0],'T-Mobile':[255,0,238],'Verizon':[255,21,0]}
for index,carrier in enumerate(carrierDF.Carrier.unique()):
    if not carrier in lvlColor:
        #randx=random.randint(0,255)
        #randy=random.randint(0,255)
        #randz=random.randint(0,255)
        #lvlColor[carrier]=(randx, randy, randz)
        lvlColor[carrier]=(carrierColors[carrier][0], carrierColors[carrier][1], carrierColors[carrier][2])

In [3]:
uniqueCarriers=carrierDF.Carrier.unique()
uniqueManufactures=carrierDF.Manufacture.unique()

lvlColor={}
carrierColors={'AT&T':[64, 190, 245],'Sprint':[255,162,0],'T-Mobile':[255,0,238],'Verizon':[255,21,0]}
print(uniqueCarriers)
for index,carrier in enumerate(uniqueCarriers):
    if not carrier in lvlColor:
        lvlColor[carrier]=(carrierColors[carrier][0], carrierColors[carrier][1], carrierColors[carrier][2])

manufactureColors={'BlackBerry':[38, 38, 38],'HTC':[189, 15, 108],'LG':[245, 17, 108],'Motorola':[1, 11, 120],
                   'Samsung':[10, 63, 148],'Google':[137, 214, 60],'Huawei':[191, 11, 11],
                   'Kyocera':[245, 118, 118],'ZTE':[102, 255, 247],'Essential':[20, 10, 46],
                   'ASUS':[12, 55, 59],'OnePlus':[227, 34, 73],'Alcatel':[0, 188, 217],
                   'RED':[219, 0, 0],'Razer':[219, 0, 0],'CAT':[255, 179, 0],'Coolpad':[237, 150, 0],
                   'Slate':[50, 0, 61],"Sonim":[219, 0, 22],"Sony":[23, 23, 23],'Revvl(T-Mobile)':[255,0,238],
                   "Nokia":[14, 0, 171],'Orbic':[139, 214, 0],"Palm":[255,21,0]}
print(uniqueManufactures)
# colors to work with: https://www.google.com/search?q=rgb+color+picker&oq=rgb+colo&aqs=chrome.0.69i59j0j69i57j0l3.1905j1j7&sourceid=chrome&ie=UTF-8
for index,manufacture in enumerate(uniqueManufactures):
    if not manufacture in lvlColor:
        lvlColor[manufacture]=(manufactureColors[manufacture][0], manufactureColors[manufacture][1], manufactureColors[manufacture][2])
        
        

['AT&T' 'Sprint' 'T-Mobile']
['ASUS' 'Alcatel' 'BlackBerry' 'Google' 'HTC' 'Huawei' 'Kyocera' 'LG'
 'Motorola' 'RED' 'Razer' 'Samsung' 'ZTE' 'CAT' 'Coolpad' 'Essential'
 'Slate' 'Sonim' 'OnePlus' 'Revvl(T-Mobile)' 'Sony']


In [4]:
# Number of bulletins per year for a phone

# Calculate the dates of the first year which consists of the NEXT month from the release into the next year.
nextMonth=carrierDF.Month_Phone_Release.copy()
nextMonth=nextMonth.add(1)
nextYear=carrierDF.Year_Phone_Release.copy()
nextYear=nextYear.add(1)
for index, month in enumerate(nextMonth):
        if month > 12: # meaning a new year!
            nextMonth.iloc[index]=1
            nextYear.iloc[index]+=1


# #print(carrierDF.head())

######################################################################### 
################## SET HOW MANY MONTHS OF UPDATES HERE ################## 
######################################################################### 
#howManyMonths=12 # # For year 1 through 4...
howManyMonths=6 # For every 6 months
#howManyMonths=3 # For every 3 months after release
######################################################################### 
if howManyMonths==12:
    latencyPointsCar_total={}
    for car in carrierDF.Carrier.unique():
        latencyPointsCar_total[car]={}
        for yr in range(4):
            latencyPointsCar_total[car][yr]=0
    #For each month that is greater than 12, set to 1 and add 1 to the year
    carrierDF['First_Year_Start']=carrierDF.Year_Phone_Release.map(str)+"-"+nextMonth.map(str)+"-01"
    carrierDF['First_Year_Start']=pd.to_datetime(carrierDF['First_Year_Start'])
    carrierDF['First_Year_End']=nextYear.map(str)+"-"+nextMonth.map(str)+"-01"
    carrierDF['First_Year_End']=pd.to_datetime(carrierDF['First_Year_End'])

    carrierDF['Second_Year_Start']=nextYear.map(str)+"-"+nextMonth.map(str)+"-01"
    carrierDF['Second_Year_Start']=pd.to_datetime(carrierDF['Second_Year_Start'])
    nextYear=nextYear.copy().add(1)
    carrierDF['Second_Year_End']=nextYear.map(str)+"-"+nextMonth.map(str)+"-01"
    carrierDF['Second_Year_End']=pd.to_datetime(carrierDF['Second_Year_End'])

    carrierDF['Third_Year_Start']=nextYear.map(str)+"-"+nextMonth.map(str)+"-01"
    carrierDF['Third_Year_Start']=pd.to_datetime(carrierDF['Third_Year_Start'])
    nextYear=nextYear.copy().add(1)
    carrierDF['Third_Year_End']=nextYear.map(str)+"-"+nextMonth.map(str)+"-01"
    carrierDF['Third_Year_End']=pd.to_datetime(carrierDF['Third_Year_End'])

    carrierDF['Fourth_Year_Start']=nextYear.map(str)+"-"+nextMonth.map(str)+"-01"
    carrierDF['Fourth_Year_Start']=pd.to_datetime(carrierDF['Fourth_Year_Start'])
    nextYear=nextYear.copy().add(1)
    carrierDF['Fourth_Year_End']=nextYear.map(str)+"-"+nextMonth.map(str)+"-01"
    carrierDF['Fourth_Year_End']=pd.to_datetime(carrierDF['Fourth_Year_End'])

    importantCols=[['First_Year_Start','First_Year_End'],['Second_Year_Start','Second_Year_End'],['Third_Year_Start','Third_Year_End'],['Fourth_Year_Start','Fourth_Year_End']]
    counts=[[0,0],[0,0],[0,0],[0,0]] # Each list represents a year with the number of phones followed by overall updates
    indvCounts=[[],[],[],[]]
    latencyPoints=[[],[],[],[]]
    noUpdates=[0,0,0,0]
elif howManyMonths==6:
    latencyPointsCar_total={}
    for car in carrierDF.Carrier.unique():
        latencyPointsCar_total[car]={}
        for hfyear in range(8):
            latencyPointsCar_total[car][hfyear]=0
            
    carrierDF['Mobile_Release_Date']=carrierDF.Year_Phone_Release.map(str)+"-"+carrierDF.Month_Phone_Release.map(str)+"-01"
    carrierDF['Mobile_Release_Date']=pd.to_datetime(carrierDF['Mobile_Release_Date'])
    carrierDF['Mobile_Release_Date']=carrierDF['Mobile_Release_Date']+ pd.offsets.MonthOffset(1)
    
    importantCols=[['First_1Year_Start','First_1Year_End'],['Second_1Year_Start','Second_1Year_End'],['First_2Year_Start','First_2Year_End'],['Second_2Year_Start','Second_2Year_End'],['First_3Year_Start','First_3Year_End'],['Second_3Year_Start','Second_3Year_End'],['First_4Year_Start','First_4Year_End'],['Second_4Year_Start','Second_4Year_End']]
    
    carrierDF['First_1Year_Start']=carrierDF.Year_Phone_Release.map(str)+"-"+nextMonth.map(str)+"-01"
    carrierDF['First_1Year_Start']=pd.to_datetime(carrierDF['First_1Year_Start'])
    carrierDF['First_1Year_End']=carrierDF['First_1Year_Start'] + pd.offsets.MonthOffset(6)

    carrierDF['Second_1Year_Start']=carrierDF['First_1Year_End'].copy()
    carrierDF['Second_1Year_End']=carrierDF['Second_1Year_Start'] + pd.offsets.MonthOffset(6)

    carrierDF['First_2Year_Start']=carrierDF['Second_1Year_End'].copy()
    carrierDF['First_2Year_End']=carrierDF['First_2Year_Start'] + pd.offsets.MonthOffset(6)

    carrierDF['Second_2Year_Start']=carrierDF['First_2Year_End'].copy()
    carrierDF['Second_2Year_End']=carrierDF['Second_2Year_Start'] + pd.offsets.MonthOffset(6)
            
    carrierDF['First_3Year_Start']=carrierDF['Second_2Year_End'].copy()
    carrierDF['First_3Year_End']=carrierDF['First_3Year_Start'] + pd.offsets.MonthOffset(6)

    carrierDF['Second_3Year_Start']=carrierDF['First_3Year_End'].copy()
    carrierDF['Second_3Year_End']=carrierDF['Second_3Year_Start'] + pd.offsets.MonthOffset(6)

    carrierDF['First_4Year_Start']=carrierDF['Second_3Year_End'].copy()
    carrierDF['First_4Year_End']=carrierDF['First_4Year_Start'] + pd.offsets.MonthOffset(6)

    carrierDF['Second_4Year_Start']=carrierDF['First_4Year_End'].copy()
    carrierDF['Second_4Year_End']=carrierDF['Second_4Year_Start'] + pd.offsets.MonthOffset(6)
    
    counts=[[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]] # Each list represents a year with the number of phones followed by overall updates
    indvCounts=[[],[],[],[],[],[],[],[]]
    indvCounts_all=[[],[],[],[],[],[],[],[]]
    latencyPoints=[[],[],[],[],[],[],[],[]]
    noUpdates=[0,0,0,0,0,0,0,0]
    
elif howManyMonths==3:
    latencyPointsCar_total={}
    for car in carrierDF.Carrier.unique():
        latencyPointsCar_total[car]={}
        for hfyear in range(int((12/howManyMonths)*4)):
            latencyPointsCar_total[car][hfyear]=0
            
    carrierDF['Mobile_Release_Date']=carrierDF.Year_Phone_Release.map(str)+"-"+carrierDF.Month_Phone_Release.map(str)+"-01"
    carrierDF['Mobile_Release_Date']=pd.to_datetime(carrierDF['Mobile_Release_Date'])
    carrierDF['Mobile_Release_Date']=carrierDF['Mobile_Release_Date']+ pd.offsets.MonthOffset(1)
    
    importantCols=[['1_1Year_Start','1_1Year_End'],['2_1Year_Start','2_1Year_End'],['3_1Year_Start','3_1Year_End'],['4_1Year_Start','4_1Year_End'],
                   ['1_2Year_Start','1_2Year_End'],['2_2Year_Start','2_2Year_End'],['3_2Year_Start','3_2Year_End'],['4_2Year_Start','4_2Year_End'],
                   ['1_3Year_Start','1_3Year_End'],['2_3Year_Start','2_3Year_End'],['3_3Year_Start','3_3Year_End'],['4_3Year_Start','4_3Year_End'],
                   ['1_4Year_Start','1_4Year_End'],['2_4Year_Start','2_4Year_End'],['3_4Year_Start','3_4Year_End'],['4_4Year_Start','4_4Year_End']
                  ]
    
    for index,value in enumerate(importantCols):
        if index==0:
            carrierDF[str(value[0])]=carrierDF.Year_Phone_Release.map(str)+"-"+nextMonth.map(str)+"-01"
            carrierDF[str(value[0])]=pd.to_datetime(carrierDF[str(value[0])])
            carrierDF[str(value[1])]=carrierDF[str(value[0])] + pd.offsets.MonthOffset(3)
        else:
            carrierDF[str(value[0])]=carrierDF[importantCols[index-1][1]].copy()
            carrierDF[str(value[1])]=carrierDF[str(value[0])] + pd.offsets.MonthOffset(3)
                    
    counts=[[0,0] for _ in range(int(12/howManyMonths)*4)] # Each list represents a year with the number of phones followed by overall updates
    indvCounts=[[] for _ in range(int(12/howManyMonths)*4)]
    latencyPoints=[[] for _ in range(int(12/howManyMonths)*4)]
    noUpdates=[0 for _ in range(int(12/howManyMonths)*4)]
    
carrierDF.Bulletin_Level=pd.to_datetime(carrierDF.Bulletin_Level)
carrierDF.Release_Date=pd.to_datetime(carrierDF.Release_Date)
THE_START_OF_TIMELINE=np.argmin(carrierDF.Bulletin_Level.values)
THE_START_OF_TIMELINE=carrierDF.iloc[THE_START_OF_TIMELINE].Bulletin_Level

THE_END_OF_TIMELINE=np.argmax(carrierDF.Bulletin_Level.values)
THE_END_OF_TIMELINE=carrierDF.iloc[THE_END_OF_TIMELINE].Bulletin_Level
# print(THE_START_OF_TIMELINE)
# print(THE_END_OF_TIMELINE)

allModels=[]
allModelsperDuration={}
for yr in range(int(12/howManyMonths)*4):
    allModelsperDuration[yr]=list()

uniqueCars=carrierDF.Carrier.unique()
count=0
for model in carrierDF.Model.unique():
    tempDFs=carrierDF[carrierDF.Model==model]
    for index, year in enumerate(importantCols): # traverse through section
        yrBEG=tempDFs[year[0]].iloc[0]#.unique()[0]
        yrEND=tempDFs[year[1]].iloc[0]#.unique()[0]

        #print(str(yrBEG)+" ; "+str(yrEND))
        if yrBEG >= THE_START_OF_TIMELINE and yrEND < THE_END_OF_TIMELINE: #the particular year is fully within our timeline :)
            counts[index][1]+=1 # add a mobile device to count even if it wasn't updated within this time frame
            
            # IF WE WERE TO CHANGE HOW WE MEASURE, this would be on bulletins NOT the release dates
            #tempDF=tempDFs[(tempDFs.Release_Date >= yrBEG) & (tempDFs.Release_Date < yrEND)]# filtered release dates accordingly
            
            tempDF=tempDFs[(tempDFs.Bulletin_Level >= yrBEG) & (tempDFs.Bulletin_Level < yrEND)]
            
            actualUpdates=0
            if len(tempDF)>0: # meaning an update was reported but if DF==0, it didn't fall within that year
                for bul in tempDF.Bulletin_Level.unique(): # for all updated bulletins
                    #if pd.to_datetime(bul) >= yrBEG and pd.to_datetime(bul) < yrEND: #the bulletin was released within this timeframe
                    if True: # for the above
                        tmp=tempDF[tempDF.Bulletin_Level==bul]
                        if len(tmp)>1:
                            if len(tmp.Carrier.unique())>1:
                                for car in tmp.Carrier.unique(): 
                                    latencyPointsCar_total[car][index]=latencyPointsCar_total[car][index]+1
                            avgs=tmp.difference.mean()
                            latencyPoints[index].append(avgs)
                        else:
                            latencyPoints[index].extend(tmp.difference.values)
                            for car in tmp.Carrier.unique():
                                latencyPointsCar_total[car][index]=latencyPointsCar_total[car][index]+1
                        actualUpdates+=1 # for each update
                if actualUpdates>6:
                    print(str(yrBEG)+" : "+str(yrEND))
                    print(tempDF)
                    print()
                counts[index][0]+=actualUpdates
                indvCounts[index].append(actualUpdates)
                if actualUpdates == 0:
                    noUpdates[index]+=1
                else:
                    allModels.append(model)
                    allModelsperDuration[index].append(model)
            else:
                noUpdates[index]+=1
            indvCounts_all[index].append(actualUpdates)
            

In [5]:
if not "Verizon" in carrierDF.Carrier.values: # only output if no Verizon
    if howManyMonths == 12:
        # Calculate general stats
        print("Number of updates, Number of phones")
        print(counts)
        print("Phones Updating:")
        updatedPh=[]
        for i in range(len(noUpdates)):
            updatedPh.append(counts[i][1]-value[i])
        print(updatedPh)
        print("No Updates:")
        print(noUpdates)
        print("Number of Updates Across each Year: ")
        print(indvCounts)

        for car in carrierDF.Carrier.unique():
            print(latencyPointsCar_total[car])
        print()
        #print("Updates with Difference Across each Year: ")
        #print(latencyPoints)
        print("----------------------------")
        totals=0
        for index, values in enumerate(counts):
            avg=values[0]/values[1]
            print("Avg bulletins at year " + str(index+1) + " " + str(avg))
            totals+=avg
        print("Average bulletins per year: " + str(totals/4))
    elif howManyMonths == 6:
        # Calculate general stats
        print("Number of updates, Number of phones")
        print(counts)
        print("Phones Updating:")
        updatedPh=[]
        for i in range(len(noUpdates)):
            updatedPh.append(counts[i][1]-noUpdates[i])
        print(updatedPh)
        print("No Updates:")
        print(noUpdates)
        print("----------------------------")
        totals=0
        for index, values in enumerate(counts):
            avg=values[0]/values[1]
            print("Avg bulletins at half year " + str(int((int(index+1)/2)-.3)) + " " + str(avg))
            totals+=avg
        print("Average bulletins per 6 months: " + str(totals/int((12/howManyMonths)*4)))

    elif howManyMonths == 3:
        # Calculate general stats
        print("Number of updates, Number of phones")
        print(counts)

        print("Phones Updating:")
        updatedPh=[]
        for i,value in range(len(noUpdates)):
            updatedPh.append(counts[i][1]-value)
        print(updatedPh)
        print("Phones Not Updating:")
        print(noUpdates)

        for car in carrierDF.Carrier.unique():
            print(latencyPointsCar_total[car])
        print()
        #print("Updates with Difference Across each Year: ")
        #print(latencyPoints)
        print("----------------------------")
        totals=0
        year=0
        for index, values in enumerate(counts):
            avg=values[0]/values[1]
            print("Avg bulletins at "+str(int(index%(12/howManyMonths))+1)+" partial year " + str(year+1) + " : " + str(avg))
            totals+=avg
            if int(index%(12/howManyMonths)+1)==4:
                year+=1
        print("Average bulletins per 3 months: " + str(totals/int((12/howManyMonths)*4)))


    print("Number of Unique Models Across Frequency: ")
    print(len(set(allModels)))

if howManyMonths == 12:
    latencyPointsCar={}
    for car in carrierDF.Carrier.unique():
        latencyPointsCar[car]={}
        for yr in range(4):
            latencyPointsCar[car][yr]=list()

    freqPointsCar={}
    for car in carrierDF.Carrier.unique():
        freqPointsCar[car]={}
        for yr in range(4):
            freqPointsCar[car][yr]=list()
elif howManyMonths==6 or howManyMonths==3:
    latencyPointsCar={}
    for car in carrierDF.Carrier.unique():
        latencyPointsCar[car]={}
        for yr in range(int(12/howManyMonths)*4):
            latencyPointsCar[car][yr]=list()

    freqPointsCar={}
    freqPointsCarAllDevices={}
    for car in carrierDF.Carrier.unique():
        freqPointsCar[car]={}
        freqPointsCarAllDevices[car]={}
        for yr in range(int(12/howManyMonths)*4):
            freqPointsCar[car][yr]=list()
            freqPointsCarAllDevices[car][yr]=list()
        
count=0
# for each mobile in row (2017-07-01 TO 2019-06-01 (if year fits within this time line, we calculate accordingly))
for car in carrierDF.Carrier.unique():
    tmpCar=carrierDF[carrierDF.Carrier==car]
    for model in tmpCar.Model.unique():
        tempDFs=tmpCar[tmpCar.Model==model]
        for index, year in enumerate(importantCols): # traverse through each year
            yrBEG=tempDFs[year[0]].iloc[0]#.unique()[0]
            yrEND=tempDFs[year[1]].iloc[0]#.unique()[0]

            if yrBEG >= THE_START_OF_TIMELINE and yrEND < THE_END_OF_TIMELINE: #the particular year is fully within our timeline :)
                # filtered release dates accordingly
                #tempDF=tempDFs[(tempDFs.Release_Date >= yrBEG) & (tempDFs.Release_Date < yrEND)]
                tempDF=tempDFs[(tempDFs.Bulletin_Level >= yrBEG) & (tempDFs.Bulletin_Level < yrEND)]
                
                actualUpdates=0
                if len(tempDF)>0: # meaning an update was reported but if DF==0, it didn't fall within that year
                    for bul in tempDF.Bulletin_Level.unique():
                        #if pd.to_datetime(bul) >= yrBEG and pd.to_datetime(bul) < yrEND:
                        if True: # comment this out if we filter OG by release date and not bulletin
                            tmp=tempDF[tempDF.Bulletin_Level==bul]
                            if len(tmp)>1:
                                print("how")
                            if len(tmp)>0:
                                actualUpdates+=1 # for each update  
                                latencyPointsCar[car][index].extend(tmp.difference.values)
                    if actualUpdates>0:
                        freqPointsCar[car][index].append(actualUpdates)
                freqPointsCarAllDevices[car][index].append(actualUpdates) # this includes devices within the timeframe but maybe no updates
               
    

Number of updates, Number of phones
[[213, 120], [210, 116], [177, 102], [167, 111], [111, 100], [84, 93], [9, 81], [6, 63]]
Phones Updating:
[85, 88, 64, 66, 42, 29, 8, 5]
No Updates:
[35, 28, 38, 45, 58, 64, 73, 58]
----------------------------
Avg bulletins at half year 0 1.775
Avg bulletins at half year 0 1.8103448275862069
Avg bulletins at half year 1 1.7352941176470589
Avg bulletins at half year 1 1.5045045045045045
Avg bulletins at half year 2 1.11
Avg bulletins at half year 2 0.9032258064516129
Avg bulletins at half year 3 0.1111111111111111
Avg bulletins at half year 3 0.09523809523809523
Average bulletins per 6 months: 1.1305898078173235
Number of Unique Models Across Frequency: 
159


In [6]:
# Plot latency points per update within year 1, 2, 3 etc...
data=[]
medians=[]
# for each severity, calculate the counts on each bulletin
for yr in range(int(12/howManyMonths)*4):
    print(yr)
    print(statistics.mean(latencyPoints[yr]))
    medians.append(statistics.median(latencyPoints[yr]))
    print()
    formatGood="th"
    if yr==0:
        formatGood="st"
    elif yr==1:
        formatGood="nd"
    elif yr==2:
        formatGood="rd"

    data.append(go.Box(
            y=latencyPoints[yr],
            name=str(yr+1) +formatGood, #+" period"
            boxpoints = 'all',
            showlegend=False,
            marker=dict(color="rgb(31, 119, 180)")
        ))


# Edit the layout
layout = dict(#title = "Average Latency per 6 Month Periods after Release",
              yaxis = dict(title = 'Days',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              #xaxis = dict(title = '6 Month Period'),
              plot_bgcolor='rgba(0,0,0,0)',
              font=dict(size=20))

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='avg-latency-since-release-years.png')
# print("REMEMBER TO ADD VERIZON")
# print("REMEMBER TO ADD VERIZON")
# print("REMEMBER TO ADD VERIZON")

if "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
    pio.write_image(fig, 'avg-latency-since-release-years.pdf', width=1000, height=600)
    
    
############################################################################################################################
############################################################################################################################
############################################################################################################################

# LINE CHART INSTEAD
data=[]

quarters=[]
for y in range(len(medians)):
    x=y+1
    if x==1: quarters.append("1st")
    elif x==2: quarters.append("2nd")
    elif x==3: quarters.append("3rd")
    else: quarters.append(str(x) + "th")


data.append(go.Scatter(
            x=quarters,
            y=medians,
        ))


    
# Edit the layout
layout = dict(#title = "Number of Updates per 6 Month Period after Release per Carrier",
              yaxis = dict(title = 'Median Latency of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)',range=[0,25]),
              #xaxis = dict(title = '6 Month Period'),
              plot_bgcolor='rgba(0,0,0,0)',
              boxmode='group',
              violinmode='group',
              legend_orientation="h",
              legend=dict(x=0.25, y=1.1),
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='release-years-frequency-per-carrier-median.png')


if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
    pio.write_image(fig, 'carrier-number-updates-released-per-year.pdf', width=1000, height=600)


0
18.927230046948356

1
21.79206349206349

2
21.27024482109228

3
22.74750499001996

4
21.01801801801802

5
23.886904761904763

6
12.777777777777779

7
9.0



In [7]:

data=[]
# for each severity, calculate the counts on each bulletin
medians=[]


for yr in range(int(12/howManyMonths)*4): # create the line chart
    formatGood="th"
    if yr==0:
        formatGood="st"
    elif yr==1:
        formatGood="nd"
    elif yr==2:
        formatGood="rd"

    medians.append(statistics.median(indvCounts_all[yr]))
    data.append(go.Box(
        y=indvCounts_all[yr],
        name=str(yr+1) +formatGood, #+" period"
        boxpoints = 'all',
        showlegend=False,
        marker=dict(color="rgb(31, 119, 180)")
    ))

# Edit the layout
layout = dict(#title = "Number of Updates after Release",
              yaxis = dict(title = 'Number of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              #xaxis = dict(title = '6 Month Period '),
              plot_bgcolor='rgba(0,0,0,0)',
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='frequency-since-release-years.png')


if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
    pio.write_image(fig, 'frequency-since-release-years.pdf', width=1000, height=600)

    
############################################################################################################################
############################################################################################################################
############################################################################################################################

# LINE CHART INSTEAD
data=[]
quarters=[]
for y in range(len(medians)):
    x=y+1
    if x==1: quarters.append("1st")
    elif x==2: quarters.append("2nd")
    elif x==3: quarters.append("3rd")
    else: quarters.append(str(x) + "th")


data.append(go.Scatter(
            x=quarters,
            y=medians,
            marker=dict(color="rgb(31, 119, 180)")
        ))


    
# Edit the layout
layout = dict(#title = "Number of Updates per 6 Month Period after Release per Carrier",
              yaxis = dict(title = 'Median Frequency of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)',range=[0,2]),
              #xaxis = dict(title = '6 Month Period'),
              plot_bgcolor='rgba(0,0,0,0)',
              boxmode='group',
              violinmode='group',
              legend_orientation="h",
              legend=dict(x=0.25, y=1.1),
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='release-years-frequency-per-carrier-median.png')


# if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
#     pio.write_image(fig, 'carrier-number-updates-released-per-year.pdf', width=1000, height=600)


    

In [8]:
# exception here

data=[]
total_updates={}
total_years={}

total_updates_wZeroes={}
total_years_wZeroes={}


for yr in range(int(12/howManyMonths)*4):

    properList_update=[]
    properList_years=[]
    properList_update_wZeroes=[]
    properList_years_wZeroes=[]
    for car in carrierDF.Carrier.unique():
        properList_update_wZeroes.extend(freqPointsCarAllDevices[car][yr])
        properList_years_wZeroes.extend([car for _ in range(len(freqPointsCarAllDevices[car][yr]))]) #+" period"
    
    total_updates_wZeroes[yr]=properList_update_wZeroes.copy()
    total_years_wZeroes[yr]=properList_years_wZeroes.copy()
    

for yr in range(int(12/howManyMonths)*4):
        formatGood="th"
        if yr==0:
            formatGood="st"
        elif yr==1:
            formatGood="nd"
        elif yr==2:
            formatGood="rd"
            
        data.append(go.Box(
            x=total_years_wZeroes[yr],
            y=total_updates_wZeroes[yr],
            name=str(yr+1)+formatGood,
#             marker=dict(color='rgb'+str(lvlColor[car]))
        ))
    
    
# Edit the layout
layout = dict(#title = "Number of Updates per 6 Month Period after Release per Carrier",
              yaxis = dict(title = 'Number of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              #xaxis = dict(title = '6 Month Period'),
              plot_bgcolor='rgba(0,0,0,0)',
              boxmode='group',
              violinmode='group',
              legend_orientation="h",
              legend=dict(x=0.25, y=1.1),
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='release-years-frequency-per-carrier.png')


if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
    pio.write_image(fig, 'carrier-number-updates-released-per-year.pdf', width=1000, height=600)

    
############################################################################################################################
############################################################################################################################
############################################################################################################################

# LINE CHART INSTEAD
from statistics import median
data=[]


poperCars_yr=[]
poperCars={}
for car in carrierDF.Carrier.unique():
    poperCars[car]=[]

for yr in range(int(12/howManyMonths)*4):
    poperCars_yr.append(yr)
    for car in carrierDF.Carrier.unique():
        poperCars[car].append(median(freqPointsCarAllDevices[car][yr])) # for each carrier, calculate median for ea section
        
for car in carrierDF.Carrier.unique():
    data.append(go.Scatter(
            x=poperCars_yr,
            y=poperCars[car],
            name=str(car),
            marker=dict(color='rgb'+str(lvlColor[car]))
        ))

# Edit the layout
layout = dict(#title = "Number of Updates per 6 Month Period after Release per Carrier",
              yaxis = dict(title = 'Median Number of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              #xaxis = dict(title = '6 Month Period'),
              plot_bgcolor='rgba(0,0,0,0)',
              boxmode='group',
              violinmode='group',
              legend_orientation="h",
              legend=dict(x=0.25, y=1.1),
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='release-years-frequency-per-carrier-median.png')


if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
    pio.write_image(fig, 'carrier-number-updates-released-per-year.pdf', width=1000, height=600)



In [29]:
# exception here

data=[]
total_updates={}
total_years={}

total_updates_wZeroes={}
total_years_wZeroes={}


for yr in range(int(12/howManyMonths)*4):

    properList_update=[]
    properList_years=[]
    properList_update_wZeroes=[]
    properList_years_wZeroes=[]
    for car in carrierDF.Carrier.unique():
        properList_update_wZeroes.extend(freqPointsCarAllDevices[car][yr])
        properList_years_wZeroes.extend([car for _ in range(len(freqPointsCarAllDevices[car][yr]))]) #+" period"
    
    total_updates_wZeroes[yr]=properList_update_wZeroes.copy()
    total_years_wZeroes[yr]=properList_years_wZeroes.copy()
    
total_x=[]
total_y=[]
for yr in range(int(12/howManyMonths)*4):
    total_x.extend(total_years_wZeroes[yr])
    total_y.extend(total_updates_wZeroes[yr])
            
data.append(go.Box(
    x=total_x,
    y=total_y,
    marker=dict(color="rgb(31, 119, 180)")
))
    
    
# Edit the layout
layout = dict(#title = "Number of Updates per 6 Month Period after Release per Carrier",
              yaxis = dict(title = 'Updates per 6-Months',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              #xaxis = dict(title = '6 Month Period'),
              plot_bgcolor='rgba(0,0,0,0)',
              boxmode='group',
              violinmode='group',
              legend_orientation="h",
              legend=dict(x=0.25, y=1.1),
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='release-years-frequency-per-carrier.png')


if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
    pio.write_image(fig, 'carrier-number-updates-released-per-year-CARONLY.pdf', width=1000, height=600)


In [9]:
# import statistics

# if howManyMonths == 12:
#     latencyPointsCar={}
#     for car in carrierDF.Carrier.unique():
#         latencyPointsCar[car]={}
#         for yr in range(4):
#             latencyPointsCar[car][yr]=list()

#     freqPointsCar={}
#     for car in carrierDF.Carrier.unique():
#         freqPointsCar[car]={}
#         for yr in range(4):
#             freqPointsCar[car][yr]=list()


if howManyMonths==6 or howManyMonths==3:
    latencyPointsMan={}
    freqPointsManAllDevices={}
    for man in carrierDF.Manufacture.unique():
        latencyPointsMan[man]={}
        freqPointsManAllDevices[man]={}
        for yr in range(int(12/howManyMonths)*4):
            latencyPointsMan[man][yr]=list()
            freqPointsManAllDevices[man][yr]=list()

    freqPointsMan={}
    for man in carrierDF.Manufacture.unique():
        freqPointsMan[man]={}
        for yr in range(int(12/howManyMonths)*4):
            freqPointsMan[man][yr]=list()
        
count=0
# for each mobile in row (2017-07-01 TO 2019-06-01 (if year fits within this time line, we calculate accordingly))
for man in carrierDF.Manufacture.unique():
    tmpMan=carrierDF[carrierDF.Manufacture==man]
    for model in tmpMan.Model.unique():
        tempDFs=tmpMan[tmpMan.Model==model]
        for index, year in enumerate(importantCols): # traverse through each year
            yrBEG=tempDFs[year[0]].iloc[0]#.unique()[0]
            yrEND=tempDFs[year[1]].iloc[0]#.unique()[0]

            if yrBEG >= THE_START_OF_TIMELINE and yrEND < THE_END_OF_TIMELINE: #the particular year/session is fully within our timeline :)
                # filtered release dates accordingly
                #tempDF=tempDFs[(tempDFs.Release_Date >= yrBEG) & (tempDFs.Release_Date < yrEND)]
                tempDF=tempDFs[(tempDFs.Bulletin_Level >= yrBEG) & (tempDFs.Bulletin_Level < yrEND)]
                
                actualUpdates=0
                if len(tempDF)>0: # meaning an update was reported but if DF==0, it didn't fall within that year
                    for bul in tempDF.Bulletin_Level.unique():
#                         if pd.to_datetime(bul) >= yrBEG and pd.to_datetime(bul) < yrEND:
                        if True:
                            tmp=tempDF[tempDF.Bulletin_Level==bul]
                            if len(tmp)>1: #calculate the average
                                actualUpdates+=1 # for each update  
                                latencyPointsMan[man][index].append(statistics.mean(tmp.difference.values))
                            elif len(tmp)==1:
                                actualUpdates+=1 # for each update  
                                latencyPointsMan[man][index].extend(tmp.difference.values)
                        
                    if actualUpdates>0:
                        freqPointsMan[man][index].append(actualUpdates)
                freqPointsManAllDevices[man][index].append(actualUpdates) # this includes devices within the timeframe but maybe no updates
                

In [10]:
data=[]
total_updates={}
total_years={}
total_updates_wZeroes={}
total_man_wZeroes={}


normedMan=[]
for man in carrierDF.Manufacture.unique():
    temp=carrierDF[carrierDF.Manufacture==man]
    if len(temp.Model.unique())>2: # only include manfuactures with 3 or more phone models
        normedMan.append(man)
importantMan=list(set(normedMan)) # get a list of important manufacturers
    
# sort important manufactures based on number of updates
manLengths=[]
for man in importantMan:
    manLengths.append(len(carrierDF[carrierDF.Manufacture==man]))
importantMan = [x for _,x in sorted(zip(manLengths,importantMan),reverse=True)]
   

for yr in range(int(12/howManyMonths)*4):# for each year range,
    properList_update_wZeroes=[]
    properList_manu_wZeroes=[]
    for man in importantMan:# for each manufacturer
        properList_update_wZeroes.extend(freqPointsManAllDevices[man][yr])
        properList_manu_wZeroes.extend([man for _ in range(len(freqPointsManAllDevices[man][yr]))]) # +" period"   
    total_updates_wZeroes[yr]=properList_update_wZeroes.copy()
    total_man_wZeroes[yr]=properList_manu_wZeroes.copy()
    
for yr in range(int(12/howManyMonths)*4): # per each year
    formatGood="th"
    if yr==0:
        formatGood="st"
    elif yr==1:
        formatGood="nd"
    elif yr==2:
        formatGood="rd"
        
    data.append(go.Box(
            x=total_man_wZeroes[yr], # needs to be each manufacturer
            y=total_updates_wZeroes[yr],
            name=str(yr+1)+formatGood,
            #marker=dict(color='rgb'+str(lvlColor[man]))
        ))
    
# Edit the layout
colorway=['rgb(60, 180, 75)', 'rgb(230, 25, 75)', 'rgb(0, 130, 200)', 'rgb(245, 130, 48)', 'rgb(145, 30, 180)', 'rgb(240, 50, 230)', 'rgb(0, 128, 128)', 'rgb(170, 110, 40)', 'rgb(0, 130, 200)', 'rgb(128, 128, 128)','rgb(128, 0, 0)', 'rgb(170, 255, 195)', 'rgb(128, 128, 0)', 'rgb(255, 215, 180)', 'rgb(0, 0, 128)', 'rgb(128, 128, 128)', 'rgb(255, 255, 255)','rgb(230, 25, 75)',  'rgb(0, 0, 0)','rgb(255, 225, 25)',  'rgb(255, 250, 200)', 'rgb(250, 190, 190)']
layout = dict(#title = "Number of True Updates per 6 Month Period after Release per Manufacture ",
              yaxis = dict(title = 'Number of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              #xaxis = dict(title = '6 Month Period',showgrid=True,zeroline=True,showline=True,zerolinecolor='#969696'),
              plot_bgcolor='rgba(0,0,0,0)',
              boxmode='group',
              colorway=colorway,
              legend_orientation="h",
              legend=dict(x=0.24, y=1.15), # for pt .33
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='release-years-frequency-per-carrier.png')


if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
    pio.write_image(fig, 'manufacture-freq.pdf', width=1100, height=600)

In [11]:
data=[]
total_updates={}
total_years={}
total_updates_wZeroes={}
total_man_wZeroes={}


print(len(carrierDF.Manufacture.unique()))
normedMan=[]
for man in carrierDF.Manufacture.unique():
    normedMan.append(man)
importantMan=list(set(normedMan)) # get a list of important manufacturers
    
# sort important manufactures based on number of updates
manLengths=[]
for man in importantMan:
    manLengths.append(len(carrierDF[carrierDF.Manufacture==man]))
importantMan = [x for _,x in sorted(zip(manLengths,importantMan),reverse=True)]
   

for yr in range(int(12/howManyMonths)*4):# for each year range,
    properList_update_wZeroes=[]
    properList_manu_wZeroes=[]
    for man in importantMan:# for each manufacturer
        properList_update_wZeroes.extend(freqPointsManAllDevices[man][yr])
        properList_manu_wZeroes.extend([man for _ in range(len(freqPointsManAllDevices[man][yr]))]) # +" period"   
    total_updates_wZeroes[yr]=properList_update_wZeroes.copy()
    total_man_wZeroes[yr]=properList_manu_wZeroes.copy()
    
for yr in range(int(12/howManyMonths)*4): # per each year
    formatGood="th"
    if yr==0:
        formatGood="st"
    elif yr==1:
        formatGood="nd"
    elif yr==2:
        formatGood="rd"
        
    data.append(go.Box(
            x=total_man_wZeroes[yr], # needs to be each manufacturer
            y=total_updates_wZeroes[yr],
            name=str(yr+1)+formatGood,
            #marker=dict(color='rgb'+str(lvlColor[man]))
        ))
    
# Edit the layout
colorway=['rgb(60, 180, 75)', 'rgb(230, 25, 75)', 'rgb(0, 130, 200)', 'rgb(245, 130, 48)', 'rgb(145, 30, 180)', 'rgb(240, 50, 230)', 'rgb(0, 128, 128)', 'rgb(170, 110, 40)', 'rgb(0, 130, 200)', 'rgb(128, 128, 128)','rgb(128, 0, 0)', 'rgb(170, 255, 195)', 'rgb(128, 128, 0)', 'rgb(255, 215, 180)', 'rgb(0, 0, 128)', 'rgb(128, 128, 128)', 'rgb(255, 255, 255)','rgb(230, 25, 75)',  'rgb(0, 0, 0)','rgb(255, 225, 25)',  'rgb(255, 250, 200)', 'rgb(250, 190, 190)']
layout = dict(#title = "Number of True Updates per 6 Month Period after Release per Manufacture ",
              yaxis = dict(title = 'Number of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              #xaxis = dict(title = '6 Month Period',showgrid=True,zeroline=True,showline=True,zerolinecolor='#969696'),
              plot_bgcolor='rgba(0,0,0,0)',
              boxmode='group',
              colorway=colorway,
              legend_orientation="h",
              legend=dict(x=0.24, y=1.15), # for pt .33
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='release-years-frequency-per-carrier.png')

print("REMEMBER TO TAKE OUT VERIZON")
print("REMEMBER TO TAKE OUT VERIZON")
print("REMEMBER TO TAKE OUT VERIZON")

if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
    pio.write_image(fig, 'manufacture-freq-all.pdf', width=1100, height=600)

21


REMEMBER TO TAKE OUT VERIZON
REMEMBER TO TAKE OUT VERIZON
REMEMBER TO TAKE OUT VERIZON


In [12]:
# Get max of each..
data=[]
total_updates={}
total_years={}
total_updates_wZeroes={}
total_man_wZeroes={}

#remove manufactures with only two or less phones
print(len(carrierDF.Manufacture.unique()))
normedMan=[]
for man in carrierDF.Manufacture.unique():
    temp=carrierDF[carrierDF.Manufacture==man]
    if len(temp.Model.unique())>2: # only include manfuactures with 3 or more phone models
        normedMan.append(man)
importantMan=list(set(normedMan)) # get a list of important manufacturers
    
# sort important manufactures based on number of updates
manLengths=[]
for man in importantMan:
    manLengths.append(len(carrierDF[carrierDF.Manufacture==man]))
importantMan = [x for _,x in sorted(zip(manLengths,importantMan),reverse=True)]
   

for yr in range(int(12/howManyMonths)*4):# for each year range,
    properList_update_wZeroes=[]
    properList_manu_wZeroes=[]
    for man in importantMan:# for each manufacturer
        #for calculating median
        #properList_update_wZeroes.extend(freqPointsManAllDevices[man][yr])
        #properList_manu_wZeroes.extend([man for _ in range(len(freqPointsManAllDevices[man][yr]))]) # +" period"  
        
        # For the max value
        if len(freqPointsManAllDevices[man][yr])>0:
            properList_update_wZeroes.append(max(freqPointsManAllDevices[man][yr]))
            properList_manu_wZeroes.append(man)
        
    total_updates_wZeroes[yr]=properList_update_wZeroes.copy()
    total_man_wZeroes[yr]=properList_manu_wZeroes.copy()
    
for yr in range(int(12/howManyMonths)*4): # per each year
    formatGood="th"
    if yr==0:
        formatGood="st"
    elif yr==1:
        formatGood="nd"
    elif yr==2:
        formatGood="rd"
        
#     print(yr)
#     print(total_man_wZeroes[yr])
#     print(total_updates_wZeroes[yr])
    data.append(go.Bar(
            x=total_man_wZeroes[yr], # needs to be each manufacturer
            y=total_updates_wZeroes[yr],
            name=str(yr+1)+formatGood,
            #marker=dict(color='rgb'+str(lvlColor[man]))
        ))
    
# Edit the layout
colorway=['rgb(60, 180, 75)', 'rgb(230, 25, 75)', 'rgb(0, 130, 200)', 'rgb(245, 130, 48)', 'rgb(145, 30, 180)', 'rgb(240, 50, 230)', 'rgb(0, 128, 128)', 'rgb(170, 110, 40)', 'rgb(0, 130, 200)', 'rgb(128, 128, 128)','rgb(128, 0, 0)', 'rgb(170, 255, 195)', 'rgb(128, 128, 0)', 'rgb(255, 215, 180)', 'rgb(0, 0, 128)', 'rgb(128, 128, 128)', 'rgb(255, 255, 255)','rgb(230, 25, 75)',  'rgb(0, 0, 0)','rgb(255, 225, 25)',  'rgb(255, 250, 200)', 'rgb(250, 190, 190)']
layout = dict(#title = "Number of True Updates per 6 Month Period after Release per Manufacture ",
              yaxis = dict(title = 'Number of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              #xaxis = dict(title = '6 Month Period',showgrid=True,zeroline=True,showline=True,zerolinecolor='#969696'),
              plot_bgcolor='rgba(0,0,0,0)',
              boxmode='group',
              colorway=colorway,
              legend_orientation="h",
              legend=dict(x=0.24, y=1.15), # for pt .33
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='release-years-frequency-per-carrier.png')

print("REMEMBER TO TAKE OUT VERIZON")
print("REMEMBER TO TAKE OUT VERIZON")
print("REMEMBER TO TAKE OUT VERIZON")

if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
    pio.write_image(fig, 'manufacture-freq-max.pdf', width=1100, height=600)


21


REMEMBER TO TAKE OUT VERIZON
REMEMBER TO TAKE OUT VERIZON
REMEMBER TO TAKE OUT VERIZON


In [13]:
# Only the manufacturers shared across all carriers

# Get max of each..
data=[]
total_updates={}
total_years={}
total_updates_wZeroes={}
total_man_wZeroes={}
normedMan=[]

# #remove manufactures with only two or less phones
# print(len(carrierDF.Manufacture.unique()))
# normedMan=[]
# for man in carrierDF.Manufacture.unique():
#     temp=carrierDF[carrierDF.Manufacture==man]
# #     if len(temp.Model.unique())>2: # only include manfuactures with 3 or more phone models
#     normedMan.append(man)
# importantMan=list(set(normedMan)) # get a list of important manufacturers
    
# Only include manufacturers shared across all carriers
for man in carrierDF.Manufacture.unique():
    temp=carrierDF[carrierDF.Manufacture==man]
    if len(temp.Carrier.unique())>2: # meaning the 3 we care about exists
        normedMan.append(man)
importantMan=list(set(normedMan))    
 
    
# sort important manufactures based on number of updates
manLengths=[]
for man in importantMan:
    manLengths.append(len(carrierDF[carrierDF.Manufacture==man]))
importantMan = [x for _,x in sorted(zip(manLengths,importantMan),reverse=True)]
   

for yr in range(int(12/howManyMonths)*4):# for each year range,
    properList_update_wZeroes=[]
    properList_manu_wZeroes=[]
    for man in importantMan:# for each manufacturer
        #for calculating median
        #properList_update_wZeroes.extend(freqPointsManAllDevices[man][yr])
        #properList_manu_wZeroes.extend([man for _ in range(len(freqPointsManAllDevices[man][yr]))]) # +" period"  
        
        # For the max value
        if len(freqPointsManAllDevices[man][yr])>0:
            properList_update_wZeroes.append(max(freqPointsManAllDevices[man][yr]))
            properList_manu_wZeroes.append(man)
        
    total_updates_wZeroes[yr]=properList_update_wZeroes.copy()
    total_man_wZeroes[yr]=properList_manu_wZeroes.copy()
    
for yr in range(int(12/howManyMonths)*4): # per each year
    formatGood="th"
    if yr==0:
        formatGood="st"
    elif yr==1:
        formatGood="nd"
    elif yr==2:
        formatGood="rd"
        
#     print(yr)
#     print(total_man_wZeroes[yr])
#     print(total_updates_wZeroes[yr])
    data.append(go.Bar(
            x=total_man_wZeroes[yr], # needs to be each manufacturer
            y=total_updates_wZeroes[yr],
            name=str(yr+1)+formatGood,
            #marker=dict(color='rgb'+str(lvlColor[man]))
        ))
    
# Edit the layout
colorway=['rgb(60, 180, 75)', 'rgb(230, 25, 75)', 'rgb(0, 130, 200)', 'rgb(245, 130, 48)', 'rgb(145, 30, 180)', 'rgb(240, 50, 230)', 'rgb(0, 128, 128)', 'rgb(170, 110, 40)', 'rgb(0, 130, 200)', 'rgb(128, 128, 128)','rgb(128, 0, 0)', 'rgb(170, 255, 195)', 'rgb(128, 128, 0)', 'rgb(255, 215, 180)', 'rgb(0, 0, 128)', 'rgb(128, 128, 128)', 'rgb(255, 255, 255)','rgb(230, 25, 75)',  'rgb(0, 0, 0)','rgb(255, 225, 25)',  'rgb(255, 250, 200)', 'rgb(250, 190, 190)']
layout = dict(#title = "Number of True Updates per 6 Month Period after Release per Manufacture ",
              yaxis = dict(title = 'Max Number of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              #xaxis = dict(title = '6 Month Period',showgrid=True,zeroline=True,showline=True,zerolinecolor='#969696'),
              plot_bgcolor='rgba(0,0,0,0)',
              boxmode='group',
              colorway=colorway,
              legend_orientation="h",
              legend=dict(x=0.24, y=1.15), # for pt .33
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='release-years-frequency-per-carrier.png')


if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
    pio.write_image(fig, 'manufacture-freq-shared-max.pdf', width=1100, height=600)



In [14]:
# Only the manufacturers shared across all carriers

# Get max of each..
data=[]
total_updates={}
total_years={}
total_updates_wZeroes={}
total_man_wZeroes={}
normedMan=[]

# #remove manufactures with only two or less phones
# print(len(carrierDF.Manufacture.unique()))
# normedMan=[]
# for man in carrierDF.Manufacture.unique():
#     temp=carrierDF[carrierDF.Manufacture==man]
# #     if len(temp.Model.unique())>2: # only include manfuactures with 3 or more phone models
#     normedMan.append(man)
# importantMan=list(set(normedMan)) # get a list of important manufacturers
    
# Only include manufacturers shared across all carriers
for man in carrierDF.Manufacture.unique():
    temp=carrierDF[carrierDF.Manufacture==man]
    if len(temp.Carrier.unique())>2: # meaning the 3 we care about exists
        normedMan.append(man)
importantMan=list(set(normedMan))    
 
    
# sort important manufactures based on number of updates
manLengths=[]
for man in importantMan:
    manLengths.append(len(carrierDF[carrierDF.Manufacture==man]))
importantMan = [x for _,x in sorted(zip(manLengths,importantMan),reverse=True)]
   

for yr in range(int(12/howManyMonths)*4):# for each year range,
    properList_update_wZeroes=[]
    properList_manu_wZeroes=[]
    for man in importantMan:# for each manufacturer
        #for calculating median
        #properList_update_wZeroes.extend(freqPointsManAllDevices[man][yr])
        #properList_manu_wZeroes.extend([man for _ in range(len(freqPointsManAllDevices[man][yr]))]) # +" period"  
        
        # For the max value
        if len(freqPointsManAllDevices[man][yr])>0:
            properList_update_wZeroes.append(statistics.median(freqPointsManAllDevices[man][yr]))
            properList_manu_wZeroes.append(man)
        
    total_updates_wZeroes[yr]=properList_update_wZeroes.copy()
    total_man_wZeroes[yr]=properList_manu_wZeroes.copy()
    
for yr in range(int(12/howManyMonths)*4): # per each year
    formatGood="th"
    if yr==0:
        formatGood="st"
    elif yr==1:
        formatGood="nd"
    elif yr==2:
        formatGood="rd"
        
#     print(yr)
#     print(total_man_wZeroes[yr])
#     print(total_updates_wZeroes[yr])
    data.append(go.Bar(
            x=total_man_wZeroes[yr], # needs to be each manufacturer
            y=total_updates_wZeroes[yr],
            name=str(yr+1)+formatGood,
            #marker=dict(color='rgb'+str(lvlColor[man]))
        ))
    
# Edit the layout
colorway=['rgb(60, 180, 75)', 'rgb(230, 25, 75)', 'rgb(0, 130, 200)', 'rgb(245, 130, 48)', 'rgb(145, 30, 180)', 'rgb(240, 50, 230)', 'rgb(0, 128, 128)', 'rgb(170, 110, 40)', 'rgb(0, 130, 200)', 'rgb(128, 128, 128)','rgb(128, 0, 0)', 'rgb(170, 255, 195)', 'rgb(128, 128, 0)', 'rgb(255, 215, 180)', 'rgb(0, 0, 128)', 'rgb(128, 128, 128)', 'rgb(255, 255, 255)','rgb(230, 25, 75)',  'rgb(0, 0, 0)','rgb(255, 225, 25)',  'rgb(255, 250, 200)', 'rgb(250, 190, 190)']
layout = dict(#title = "Number of True Updates per 6 Month Period after Release per Manufacture ",
              yaxis = dict(title = 'Median Number of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              #xaxis = dict(title = '6 Month Period',showgrid=True,zeroline=True,showline=True,zerolinecolor='#969696'),
              plot_bgcolor='rgba(0,0,0,0)',
              boxmode='group',
              colorway=colorway,
              legend_orientation="h",
              legend=dict(x=0.24, y=1.15), # for pt .33
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='release-years-frequency-per-carrier.png')


if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
    pio.write_image(fig, 'manufacture-freq-shared-median.pdf', width=1100, height=600)



In [15]:
data=[]
total_updates={}
total_years={}
total_updates_wZeroes={}
total_man_wZeroes={}


normedMan=[]
for man in carrierDF.Manufacture.unique():
    temp=carrierDF[carrierDF.Manufacture==man]
    if len(temp.Carrier.unique())>2: # meaning the 3 we care about exists
        normedMan.append(man)
importantMan=list(set(normedMan)) # get a list of important manufacturers
    
# sort important manufactures based on number of updates
manLengths=[]
for man in importantMan:
    manLengths.append(len(carrierDF[carrierDF.Manufacture==man]))
importantMan = [x for _,x in sorted(zip(manLengths,importantMan),reverse=True)]
   

for yr in range(int(12/howManyMonths)*4):# for each year range,
    properList_update_wZeroes=[]
    properList_manu_wZeroes=[]
    for man in importantMan:# for each manufacturer
        properList_update_wZeroes.extend(freqPointsManAllDevices[man][yr])
        properList_manu_wZeroes.extend([man for _ in range(len(freqPointsManAllDevices[man][yr]))]) # +" period"   
    total_updates_wZeroes[yr]=properList_update_wZeroes.copy()
    total_man_wZeroes[yr]=properList_manu_wZeroes.copy()
    
for yr in range(int(12/howManyMonths)*4): # per each year
    formatGood="th"
    if yr==0:
        formatGood="st"
    elif yr==1:
        formatGood="nd"
    elif yr==2:
        formatGood="rd"
        
    data.append(go.Box(
            x=total_man_wZeroes[yr], # needs to be each manufacturer
            y=total_updates_wZeroes[yr],
            name=str(yr+1)+formatGood,
            #marker=dict(color='rgb'+str(lvlColor[man]))
        ))
    
# Edit the layout
colorway=['rgb(60, 180, 75)', 'rgb(230, 25, 75)', 'rgb(0, 130, 200)', 'rgb(245, 130, 48)', 'rgb(145, 30, 180)', 'rgb(240, 50, 230)', 'rgb(0, 128, 128)', 'rgb(170, 110, 40)', 'rgb(0, 130, 200)', 'rgb(128, 128, 128)','rgb(128, 0, 0)', 'rgb(170, 255, 195)', 'rgb(128, 128, 0)', 'rgb(255, 215, 180)', 'rgb(0, 0, 128)', 'rgb(128, 128, 128)', 'rgb(255, 255, 255)','rgb(230, 25, 75)',  'rgb(0, 0, 0)','rgb(255, 225, 25)',  'rgb(255, 250, 200)', 'rgb(250, 190, 190)']
layout = dict(#title = "Number of True Updates per 6 Month Period after Release per Manufacture ",
              yaxis = dict(title = 'Number of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              #xaxis = dict(title = '6 Month Period',showgrid=True,zeroline=True,showline=True,zerolinecolor='#969696'),
              plot_bgcolor='rgba(0,0,0,0)',
              boxmode='group',
              colorway=colorway,
              legend_orientation="h",
              legend=dict(x=0.24, y=1.15), # for pt .33
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='release-years-frequency-per-carrier.png')

print("REMEMBER TO TAKE OUT VERIZON")
print("REMEMBER TO TAKE OUT VERIZON")
print("REMEMBER TO TAKE OUT VERIZON")

if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
    pio.write_image(fig, 'manufacture-freq-shared.pdf', width=1100, height=600)

REMEMBER TO TAKE OUT VERIZON
REMEMBER TO TAKE OUT VERIZON
REMEMBER TO TAKE OUT VERIZON


In [25]:
data=[]
total_updates={}
total_years={}
total_updates_wZeroes={}
total_man_wZeroes={}


normedMan=[]
for man in carrierDF.Manufacture.unique():
    temp=carrierDF[carrierDF.Manufacture==man]
    if len(temp.Carrier.unique())>2: # meaning the 3 we care about exists
        normedMan.append(man)
importantMan=list(set(normedMan)) # get a list of important manufacturers
    
# sort important manufactures based on number of updates
manLengths=[]
for man in importantMan:
    manLengths.append(len(carrierDF[carrierDF.Manufacture==man]))
importantMan = [x for _,x in sorted(zip(manLengths,importantMan),reverse=True)]

    
for yr in range(int(12/howManyMonths)*4):# for each year range,
    properList_update_wZeroes=[]
    properList_manu_wZeroes=[]
    for man in importantMan:# for each manufacturer
        properList_update_wZeroes.extend(freqPointsManAllDevices[man][yr])
        properList_manu_wZeroes.extend([man for _ in range(len(freqPointsManAllDevices[man][yr]))]) # +" period"   
    total_updates_wZeroes[yr]=properList_update_wZeroes.copy()
    total_man_wZeroes[yr]=properList_manu_wZeroes.copy()

    
eaMan_x=[]
eaMan_y=[]
for yr in range(int(12/howManyMonths)*4): # per each year
    eaMan_x.extend(total_man_wZeroes[yr])
    eaMan_y.extend(total_updates_wZeroes[yr])
    
  
    
        
data.append(go.Box(
        x=eaMan_x, # needs to be each manufacturer
        y=eaMan_y,
        marker=dict(color="rgb(31, 119, 180)")
    ))
    
# Edit the layout
colorway=['rgb(60, 180, 75)', 'rgb(230, 25, 75)', 'rgb(0, 130, 200)', 'rgb(245, 130, 48)', 'rgb(145, 30, 180)', 'rgb(240, 50, 230)', 'rgb(0, 128, 128)', 'rgb(170, 110, 40)', 'rgb(0, 130, 200)', 'rgb(128, 128, 128)','rgb(128, 0, 0)', 'rgb(170, 255, 195)', 'rgb(128, 128, 0)', 'rgb(255, 215, 180)', 'rgb(0, 0, 128)', 'rgb(128, 128, 128)', 'rgb(255, 255, 255)','rgb(230, 25, 75)',  'rgb(0, 0, 0)','rgb(255, 225, 25)',  'rgb(255, 250, 200)', 'rgb(250, 190, 190)']
layout = dict(#title = "Number of True Updates per 6 Month Period after Release per Manufacture ",
              yaxis = dict(title = 'Updates per 6-Months',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              #xaxis = dict(title = '6 Month Period',showgrid=True,zeroline=True,showline=True,zerolinecolor='#969696'),
              plot_bgcolor='rgba(0,0,0,0)',
              boxmode='group',
              colorway=colorway,
              legend_orientation="h",
              legend=dict(x=0.24, y=1.15), # for pt .33
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='release-years-frequency-per-carrier.png')

print("REMEMBER TO TAKE OUT VERIZON")
print("REMEMBER TO TAKE OUT VERIZON")
print("REMEMBER TO TAKE OUT VERIZON")

if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
    pio.write_image(fig, 'manufacture-freq-perman.pdf', width=1100, height=600)

REMEMBER TO TAKE OUT VERIZON
REMEMBER TO TAKE OUT VERIZON
REMEMBER TO TAKE OUT VERIZON


In [17]:
# # For each model, calculate the frequency of updates within the first 4 years of it's release

# # Get same models across carriers
# sameModel=[]
# sameManu=[]
# for model in carrierDF.Model.unique():
#     tempsDF=carrierDF[carrierDF.Model==model]
#     if len(tempsDF.Carrier.unique())==3: # all three carriers 
#         sameModel.append(model)
#         man=list(tempsDF.Manufacture.unique())[0]
#         sameManu.append(list(tempsDF.Manufacture.unique())[0])
        
        
# print(len(sameModel))
# print(sameModel)



# frequencyPairs_perMode={}
# frequencyPairs_perMode_x={}
# for car in carrierDF.Carrier.unique():
#     frequencyPairs_perMode[car]={}
#     frequencyPairs_perMode_x[car]={}
#     for man in sameManu:
#         frequencyPairs_perMode[car][man]=list()
#         frequencyPairs_perMode_x[car][man]=list()
            
# for car in carrierDF.Carrier.unique():
#     for model in sameModel:
#         tempDFs=carrierDF[(carrierDF.Carrier==car) & (carrierDF.Model==model)]
#         man=list(tempDFs.Manufacture.unique())[0]
#         for index, year in enumerate(importantCols): # traverse through each year
#             yrBEG=tempDFs[year[0]].iloc[0]#.unique()[0]
#             yrEND=tempDFs[year[1]].iloc[0]#.unique()[0]

#             if yrBEG >= THE_START_OF_TIMELINE and yrEND < THE_END_OF_TIMELINE: #the particular year/session is fully within our timeline :)
#                 tempDF=tempDFs[(tempDFs.Bulletin_Level >= yrBEG) & (tempDFs.Bulletin_Level < yrEND)] # only look at bulletins released during this time

#                 actualUpdates=0
#                 if len(tempDF)>0: # meaning an update was reported but if DF==0, it didn't fall within that year
#                     for bul in tempDF.Bulletin_Level.unique():
#                         tmp=tempDF[tempDF.Bulletin_Level==bul]
#                         if len(tmp)>0: 
#                             actualUpdates+=1 # for each update  
#                 frequencyPairs_perMode[car][man].append(actualUpdates) # add the frequency to the list
#                 frequencyPairs_perMode_x[car][man].append(man) # add the frequency to the list


                
# data=[]             
# totalCarrier={}
# totalCarrier_xAxis={}
# for car in carrierDF.Carrier.unique():
#     totalCarrier[car]=list()
#     totalCarrier_xAxis[car]=list()

# for car in carrierDF.Carrier.unique():
#     tmp=[]
#     tmp_x=[]
#     for man in sameManu:
#         tmp.extend(frequencyPairs_perMode[car][man])
#         tmp_x.extend(frequencyPairs_perMode_x[car][man])
#     totalCarrier[car].extend(tmp.copy())
#     totalCarrier_xAxis[car].extend(tmp_x.copy())

    
# for car in carrierDF.Carrier.unique():
#     data.append(go.Box(
#             x=totalCarrier_xAxis[car], # needs to be each manufacturer
#             y=totalCarrier[car],
#             name=str(car),
#             marker=dict(color='rgb'+str(lvlColor[car])),
#             #boxpoints = 'all',
#         ))
    
# # Edit the layout
# colorway=['rgb(60, 180, 75)', 'rgb(230, 25, 75)', 'rgb(0, 130, 200)', 'rgb(245, 130, 48)', 'rgb(145, 30, 180)', 'rgb(240, 50, 230)', 'rgb(0, 128, 128)', 'rgb(170, 110, 40)', 'rgb(0, 130, 200)', 'rgb(128, 128, 128)','rgb(128, 0, 0)', 'rgb(170, 255, 195)', 'rgb(128, 128, 0)', 'rgb(255, 215, 180)', 'rgb(0, 0, 128)', 'rgb(128, 128, 128)', 'rgb(255, 255, 255)','rgb(230, 25, 75)',  'rgb(0, 0, 0)','rgb(255, 225, 25)',  'rgb(255, 250, 200)', 'rgb(250, 190, 190)']
# layout = dict(#title = "Number of True Updates per 6 Month Period after Release per Manufacture ",
#               yaxis = dict(title = '6-month Update Frequency',showgrid=True, gridcolor='rgb(219, 219, 219)'),
#               #xaxis = dict(title = '6 Month Period',showgrid=True,zeroline=True,showline=True,zerolinecolor='#969696'),
#               plot_bgcolor='rgba(0,0,0,0)',
#               boxmode='group',
#               colorway=colorway,
#               legend_orientation="h",
#               legend=dict(x=0.6, y=1.15), # for pt 14, 0.025
#               font=dict(size=20)
#               )

# fig = dict(data=data, layout=layout)
# py.offline.iplot(fig, filename='release-years-frequency-per-carrier.png')

# if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
#     pio.write_image(fig, 'freq-manu-carrier-pairs.pdf', width=1000, height=600)



In [18]:
# # # print(carrierDF[carrierDF.Manufacture=="Google"])
# print(carrierDF[carrierDF.Model.str.contains("G5")])

In [19]:
# # exception here

# data=[]
# total_updates={}
# total_years={}

# total_updates_wZeroes={}
# total_years_wZeroes={}


# for car in carrierDF.Carrier.unique():
#     properList_update=[]
#     properList_years=[]
#     properList_update_wZeroes=[]
#     properList_years_wZeroes=[]
#     for yr in range(int(12/howManyMonths)*4):
#         formatGood="th"
#         if yr==0:
#             formatGood="st"
#         elif yr==1:
#             formatGood="nd"
#         elif yr==2:
#             formatGood="rd"
        
        
#         properList_update.extend(freqPointsCar[car][yr])
#         properList_years.extend([str(yr+1)+formatGood for _ in range(len(freqPointsCar[car][yr]))]) #+" period"
        
#         properList_update_wZeroes.extend(freqPointsCarAllDevices[car][yr])
#         properList_years_wZeroes.extend([str(yr+1)+formatGood for _ in range(len(freqPointsCarAllDevices[car][yr]))]) #+" period"
#     total_updates[car]=properList_update.copy()
#     total_years[car]=properList_years.copy()
    
#     total_updates_wZeroes[car]=properList_update_wZeroes.copy()
#     total_years_wZeroes[car]=properList_years_wZeroes.copy()
    
    
# # for car in carrierDF.Carrier.unique():
# #     data.append(go.Box(
# #             x=total_years[car],
# #             y=total_updates[car],
# #             name=str(car),
# #             marker=dict(color='rgb'+str(lvlColor[car]))
# #         ))

# for car in carrierDF.Carrier.unique():
#     data.append(go.Box(
#             x=total_years_wZeroes[car],
#             y=total_updates_wZeroes[car],
#             name=str(car),
#             marker=dict(color='rgb'+str(lvlColor[car]))
#         ))

    
# # Edit the layout
# layout = dict(#title = "Number of Updates per 6 Month Period after Release per Carrier",
#               yaxis = dict(title = 'Number of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)'),
#               #xaxis = dict(title = '6 Month Period'),
#               plot_bgcolor='rgba(0,0,0,0)',
#               boxmode='group',
#               violinmode='group',
#               legend_orientation="h",
#               legend=dict(x=0.60, y=1.1),
#               font=dict(size=20)
#               )

# fig = dict(data=data, layout=layout)
# py.offline.iplot(fig, filename='release-years-frequency-per-carrier.png')


# # if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
# #     pio.write_image(fig, 'carrier-number-updates-released-per-year.pdf', width=1000, height=600)


In [20]:
# data=[]
# total_updates={}
# total_years={}
# total_updates_wZeroes={}
# total_years_wZeroes={}

# #remove manufactures with only two or less phones
# print(len(carrierDF.Manufacture.unique()))
# normedMan=[]
# for man in carrierDF.Manufacture.unique():
#     temp=carrierDF[carrierDF.Manufacture==man]
#     if len(temp.Model.unique())>2: # only include manfuactures with 3 or more phone models
#         normedMan.append(man)

        
# importantMan=[]
# for man in normedMan:
#     properList_update=[]
#     properList_years=[]
    
#     properList_update_wZeroes=[]
#     properList_years_wZeroes=[]
#     for yr in range(int(12/howManyMonths)*4):
#         formatGood="th"
#         if yr==0:
#             formatGood="st"
#         elif yr==1:
#             formatGood="nd"
#         elif yr==2:
#             formatGood="rd"
        
        
#         properList_update.extend(freqPointsManAllDevices[man][yr])
#         properList_years.extend([str(yr+1)+formatGood for _ in range(len(freqPointsManAllDevices[man][yr]))]) # +" period"
        
#         properList_update_wZeroes.extend(freqPointsManAllDevices[man][yr])
#         properList_years_wZeroes.extend([str(yr+1)+formatGood for _ in range(len(freqPointsManAllDevices[man][yr]))]) # +" period"
#         importantMan.append(man)
#     total_updates[man]=properList_update.copy()
#     total_years[man]=properList_years.copy()
    
#     total_updates_wZeroes[man]=properList_update_wZeroes.copy()
#     total_years_wZeroes[man]=properList_years_wZeroes.copy()
    
# importantMan=list(set(importantMan))
    
# # sort important manufactures based on number of updates
# manLengths=[]
# for man in importantMan:
#     manLengths.append(len(carrierDF[carrierDF.Manufacture==man]))
    
# print(importantMan)
# importantMan
# manLengths

# importantMan = [x for _,x in sorted(zip(manLengths,importantMan),reverse=True)]
   
# # for man in importantMan:
# #     data.append(go.Box(
# #             x=total_years[man],
# #             y=total_updates[man],
# #             name=str(man),
# #             #marker=dict(color='rgb'+str(lvlColor[man]))
# #         ))
    
# for man in importantMan: # per 6-month period instead
#     data.append(go.Box(
#             x=total_years_wZeroes[man],
#             y=total_updates_wZeroes[man],
#             name=str(man),
#             #marker=dict(color='rgb'+str(lvlColor[man]))
#         ))

# # Edit the layout
# colorway=['rgb(60, 180, 75)', 'rgb(230, 25, 75)', 'rgb(0, 130, 200)', 'rgb(245, 130, 48)', 'rgb(145, 30, 180)', 'rgb(240, 50, 230)', 'rgb(0, 128, 128)', 'rgb(170, 110, 40)', 'rgb(0, 130, 200)', 'rgb(128, 128, 128)','rgb(128, 0, 0)', 'rgb(170, 255, 195)', 'rgb(128, 128, 0)', 'rgb(255, 215, 180)', 'rgb(0, 0, 128)', 'rgb(128, 128, 128)', 'rgb(255, 255, 255)','rgb(230, 25, 75)',  'rgb(0, 0, 0)','rgb(255, 225, 25)',  'rgb(255, 250, 200)', 'rgb(250, 190, 190)']
# layout = dict(#title = "Number of True Updates per 6 Month Period after Release per Manufacture ",
#               yaxis = dict(title = 'Number of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)'),
#               #xaxis = dict(title = '6 Month Period',showgrid=True,zeroline=True,showline=True,zerolinecolor='#969696'),
#               plot_bgcolor='rgba(0,0,0,0)',
#               boxmode='group',
#               colorway=colorway,
#               legend_orientation="h",
#               legend=dict(x=0.2, y=1.15), # for pt 14, 0.025
#               font=dict(size=14)
#               )

# fig = dict(data=data, layout=layout)
# py.offline.iplot(fig, filename='release-years-frequency-per-carrier.png')

# print("REMEMBER TO TAKE OUT VERIZON")
# print("REMEMBER TO TAKE OUT VERIZON")
# print("REMEMBER TO TAKE OUT VERIZON")

# # if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
# #     pio.write_image(fig, 'test-manufacture-freq.pdf', width=1100, height=600)

In [21]:
###################### Manufacturers over time - two subplots ###################### 
####################################################################################

# data=[]
# total_updates={}
# total_years={}

# #remove manufactures with only two or less phones
# print(len(carrierDF.Manufacture.unique()))
# normedMan=[]
# for man in carrierDF.Manufacture.unique():
#     temp=carrierDF[carrierDF.Manufacture==man]
#     if len(temp.Model.unique())>2: # only include manfuactures with 3 or more phone models
#         normedMan.append(man)

        
# importantMan=[]
# for man in normedMan:
#     properList_update=[]
#     properList_years=[]
    
#     properList_update_wZeroes=[]
#     properList_years_wZeroes=[]
#     for yr in range(int(12/howManyMonths)*4):
#         formatGood="th"
#         if yr==0:
#             formatGood="st"
#         elif yr==1:
#             formatGood="nd"
#         elif yr==2:
#             formatGood="rd"
        
        
#         properList_update.extend(freqPointsManAllDevices[man][yr])
#         properList_years.extend([str(yr+1)+formatGood for _ in range(len(freqPointsManAllDevices[man][yr]))]) # +" period"
        
#         properList_update_wZeroes.extend(freqPointsManAllDevices[man][yr])
#         properList_years_wZeroes.extend([str(yr+1)+formatGood for _ in range(len(freqPointsManAllDevices[man][yr]))]) # +" period"
#         importantMan.append(man)
#     total_updates[man]=properList_update.copy()
#     total_years[man]=properList_years.copy()
    
#     total_updates_wZeroes[man]=properList_update_wZeroes.copy()
#     total_years_wZeroes[man]=properList_years_wZeroes.copy()
    
# importantMan=list(set(importantMan))
    
# # sort important manufactures based on number of updates
# manLengths=[]
# for man in importantMan:
#     manLengths.append(len(carrierDF[carrierDF.Manufacture==man]))
    
# print(importantMan)
# importantMan
# manLengths

# importantMan = [x for _,x in sorted(zip(manLengths,importantMan),reverse=True)]
   
# # for man in importantMan:
# #     data.append(go.Box(
# #             x=total_years[man],
# #             y=total_updates[man],
# #             name=str(man),
# #             #marker=dict(color='rgb'+str(lvlColor[man]))
# #         ))
    
# twoPlots=int(len(importantMan)/2)
# twoPlots=[twoPlots,len(importantMan)]
# print(twoPlots)

# from plotly.subplots import make_subplots
# # exception here
# fig = make_subplots(rows=2)

# colorway=['rgb(60, 180, 75)', 'rgb(230, 25, 75)', 'rgb(0, 130, 200)', 'rgb(245, 130, 48)', 'rgb(145, 30, 180)', 'rgb(240, 50, 230)', 'rgb(0, 128, 128)', 'rgb(170, 110, 40)', 'rgb(0, 130, 200)', 'rgb(128, 128, 128)','rgb(128, 0, 0)', 'rgb(170, 255, 195)', 'rgb(128, 128, 0)', 'rgb(255, 215, 180)', 'rgb(0, 0, 128)', 'rgb(128, 128, 128)', 'rgb(255, 255, 255)','rgb(230, 25, 75)',  'rgb(0, 0, 0)','rgb(255, 225, 25)',  'rgb(255, 250, 200)', 'rgb(250, 190, 190)']

# for index,man in enumerate(importantMan):
#     if index<twoPlots[0]: # for the first five manufactures
#         formatting=index
#         fig.add_box(x=total_years_wZeroes[man], y=total_updates_wZeroes[man], name=str(man), marker_color = colorway[index],offsetgroup=formatting, row=1, col=1)
#         #fig.add_trace(go.Box(x=total_years_wZeroes[man], y=total_updates_wZeroes[man], name=str(man)), 1, 1)
#     else:
#         formatting=index-twoPlots[0]
#         fig.add_box(x=total_years_wZeroes[man], y=total_updates_wZeroes[man], name=str(man), marker_color = colorway[index],offsetgroup=formatting, row=2, col=1)
#         #fig.add_trace(go.Box(x=total_years_wZeroes[man], y=total_updates_wZeroes[man], name=str(man)), 2, 2)
        
        
# fig.update_layout(yaxis_title='Number of Updates',boxmode='group',colorway=colorway,font=dict(size=14),legend_orientation="h",legend=dict(x=0, y=1.15))
# fig.show()

        

        

# # Edit the layout

# # layout = dict(#title = "Number of True Updates per 6 Month Period after Release per Manufacture ",
# #               yaxis = dict(title = 'Number of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)'),
# #               #xaxis = dict(title = '6 Month Period',showgrid=True,zeroline=True,showline=True,zerolinecolor='#969696'),
# #               plot_bgcolor='rgba(0,0,0,0)',
# #               boxmode='group',
# #               colorway=colorway,
# #               legend_orientation="h",
# #               legend=dict(x=0.2, y=1.15), # for pt 14, 0.025
# #               font=dict(size=14)
# #               )

# # fig = dict(data=data, layout=layout)
# # py.offline.iplot(fig, filename='release-years-frequency-per-carrier.png')

# # # print("REMEMBER TO TAKE OUT VERIZON")
# # # print("REMEMBER TO TAKE OUT VERIZON")
# # # print("REMEMBER TO TAKE OUT VERIZON")

# # # if not "Verizon" in carrierDF.Carrier.values and saveFigs==True: # only save if no Verizon
# # #     pio.write_image(fig, 'test-manufacture-freq.pdf', width=1100, height=600)



In [22]:
###################### Manufacturers over time - table ######################
#############################################################################

# # Graph the mean and standard deviation for manufactures per 6-month period

# # pandas dataframe with
#     # device manufacture
#     # device name
#     # 8 columns for each 6-month period
#     # each column would be the mean and std dev

# columnNames=['Manufacture','0','1','2','3','4','5','6','7']
# newDF=[]
# for man in importantMan:
#     manRow=[man]
#     for yr in range(int(12/6)*4):
#         if len(freqPointsMan[man][yr])==0:
#             manRow.append([0,0])
#         else:
#             manRow.append(freqPointsMan[man][yr])
#     newDF.append(manRow)

# newDF=pd.DataFrame(newDF, columns=columnNames)
# # print(newDF.head())

# colsOfInterest=["Manufacture"]
# stndOnly=["Manufacture"]
# stndOnlyfrmt=["Manufacture"]
# troubleshooting=["Manufacture"]

# for yr in range(int(12/6)*4): # per column, calculate the mean and standard dev
#     newDF[str(yr+1)+"_mean"]=newDF[str(yr)].apply(lambda x: round(statistics.mean(x),2))
#     newDF[str(yr+1)+"_strd"]=[round(statistics.stdev(x),2) if len(x)>1 else "" for x in newDF[str(yr)]]
#     newDF[str(yr+1)+"_var"]=[round(statistics.variance(x),2) if len(x)>1 else "" for x in newDF[str(yr)]]
#     newDF[str(yr+1)+"_mnANDstrd"]=newDF[str(yr+1)+"_mean"].map(str)+"±"+newDF[str(yr+1)+"_strd"].map(str)
#     newDF[str(yr+1)+"_troubleshooting"]=[len(x) for x in newDF[str(yr)]]
    
    
# #     colsOfInterest.append(str(yr)+"_mean")
# #     colsOfInterest.append(str(yr)+"_strd")
#     formatting="th"
#     if yr+1==1:
#         formatting="st"
#     elif yr+1==2:
#         formatting="nd"
#     elif yr+1==3:
#         formatting="rd"

        
#     colsOfInterest.append(str(yr+1)+"_mnANDstrd")
#     stndOnly.append(colsOfInterest[-1])
    
#     colsOfInterest.append(str(yr+1)+"_var")
#     stndOnlyfrmt.append(str(yr+1)+str(formatting))
#     troubleshooting.append(str(yr+1)+"_troubleshooting")

# # print(newDF.loc[:,colsOfInterest])
# temp=newDF.loc[:,stndOnly]
# temp.columns=stndOnlyfrmt
# print(temp)
# print(newDF.loc[:,troubleshooting])

In [23]:
# data=[]
# total_updates={}
# total_years={}


# for man in carrierDF.Manufacture.unique():
#     properList_update=[]
#     properList_years=[]
#     for yr in range(int(12/howManyMonths)*4):
#         if len(freqPointsMan[man][yr])<1:
#             properList_update.append(0)
#         else:
#             properList_update.append(statistics.mean(freqPointsMan[man][yr]))
#         properList_years.append(yr+1)
#     total_updates[man]=properList_update.copy()
#     total_years[man]=properList_years.copy()



    
# for man in carrierDF.Manufacture.unique():
#     data.append(go.Bar(
#             x=total_years[man],
#             y=total_updates[man],
#             name=str(man),
#             marker=dict(color='rgb'+str(lvlColor[man]))
#         ))

# # Edit the layout
# layout = dict(title = "Average Number of True Updates per 6 Month Period after Release per Manufacture",
#               yaxis = dict(title = 'Number of Updates per Phone Model',showgrid=True, gridcolor='rgb(219, 219, 219)'),
#               xaxis = dict(title = '6 Month Periods Since Mobile Release'),
#               plot_bgcolor='rgba(0,0,0,0)',
#               boxmode='group',
#               violinmode='group'

#               )

# fig = dict(data=data, layout=layout)
# py.offline.iplot(fig, filename='release-years-frequency-per-carrier.png')
# print("REMEMBER TO TAKE OUT VERIZON")
# print("REMEMBER TO TAKE OUT VERIZON")
# print("REMEMBER TO TAKE OUT VERIZON")